# CSC-52081-EP Lab4: Reinforcement Learning - TD Learning, QLearning and SARSA

<img src="https://raw.githubusercontent.com/jeremiedecock/polytechnique-csc-52081-ep-2026-students/refs/heads/main/assets/logo.jpg" style="float: left; width: 15%" />

[CSC-52081-EP-2026](https://moodle.ip-paris.fr/course/view.php?id=10691) Lab session #4

2019-2026 Jérémie Decock

[![Open in Google Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/jeremiedecock/polytechnique-csc-52081-ep-2026-students/blob/main/lab4_rl2_tdlearning_qlearning_sarsa.ipynb)

[![My Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/jeremiedecock/polytechnique-csc-52081-ep-2026-students/main?filepath=lab4_rl2_tdlearning_qlearning_sarsa.ipynb)

[![NbViewer](https://raw.githubusercontent.com/jupyter/design/main/logos/Badges/nbviewer_badge.svg)](https://nbviewer.jupyter.org/github/jeremiedecock/polytechnique-csc-52081-ep-2026-students/blob/main/lab4_rl2_tdlearning_qlearning_sarsa.ipynb)

[![Local](https://img.shields.io/badge/Local-Save%20As...-blue)](https://github.com/jeremiedecock/polytechnique-csc-52081-ep-2026-students/raw/main/lab4_rl2_tdlearning_qlearning_sarsa.ipynb)

## Introduction

The purpose of this lab is to introduce some classic concepts used
in reinforcement learning: *Temporal Difference Learning* (*TD Learning*), *QLearning* and *SARSA*.

As in previous labs, you can either:
- open, edit and execute the notebook in *Google Colab* following this link: https://colab.research.google.com/github/jeremiedecock/polytechnique-csc-52081-ep-2026-students/blob/main/lab4_rl2_tdlearning_qlearning_sarsa.ipynb ; this is the **recommended** choice as you have nothing to install on your computer
- open, edit and execute the notebook in *MyBinder* (if for any reason the Google Colab solution doesn't work): https://mybinder.org/v2/gh/jeremiedecock/polytechnique-csc-52081-ep-2026-students/main?filepath=lab4_rl2_tdlearning_qlearning_sarsa.ipynb
- download, edit and execute the notebook on your computer if Python3 and JupyterLab are already installed: https://github.com/jeremiedecock/polytechnique-csc-52081-ep-2026-students/raw/main/lab4_rl2_tdlearning_qlearning_sarsa.ipynb

If you work with Google Colab or MyBinder, **remember to save or download your work regularly or you may lose it!**

## Lab Submission

Please submit your completed notebook in [Moodle : "Lab 4 - Submission"](https://moodle.ip-paris.fr/mod/assign/view.php?id=185079).

### Submission Guidelines

1. **File Naming:** Rename your notebook as follows: **`firstname_lastname-04.ipynb`** where `firstname` and `lastname` match your email address. *Example: `jesse_read-04.ipynb`*
2. **Clear Output Cells:** To reduce file size (**must be under 500 KB**), clear all output cells before submitting. This includes rendered images, videos, plots, and dataframes...
   - **JupyterLab:**
     - Click **"Kernel" → "Restart Kernel and Clear Outputs of All Cells..."**
     - Then go to **"File" → "Save Notebook As..."**
   - **Google Colab:**
     - Click **"Edit" → "Clear all outputs"**
     - Then go to **"File" → "Download" → "Download.ipynb"**
   - **VSCode:**
     - Click **"Clear All Outputs"**
     - Then **save your file**
3. **Upload Your File:** Only **`.ipynb`** files are accepted.

**Note:** Bonus parts (if any) are optional, as their name suggests.


## Setup the Python environment

This notebook relies on several libraries including `gymnasium[toy-text]` (v1.0.0), `ipywidgets`, `matplotlib`, `moviepy`, `numpy`, `pandas`, `pygame`, `seaborn`, and `tqdm`.
A complete list of dependencies can be found in the following [requirements-lab4.txt](https://raw.githubusercontent.com/jeremiedecock/polytechnique-csc-52081-ep-2026-students/main/requirements-lab4.txt) file.

### If you use Google Colab

If you use Google Colab, execute the next cell to install required libraries.

In [ ]:
import sys, subprocess

def is_colab():
    return "google.colab" in sys.modules

def run_subprocess_command(cmd):
    # run the command
    process = subprocess.Popen(cmd.split(), stdout=subprocess.PIPE)
    # print the output
    for line in process.stdout:
        print(line.decode().strip())

if is_colab():
    # run_subprocess_command("apt install xvfb x11-utils")
    run_subprocess_command("pip install -r https://raw.githubusercontent.com/jeremiedecock/polytechnique-csc-52081-ep-2026-students/main/requirements-lab4-google-colab.txt")

### If you have downloaded the notebook on your computer and execute it in your own Python environment

To set up the necessary dependencies, run the following commands to establish a [Python virtual environment (venv)](https://docs.python.org/3/library/venv.html) that includes all the essential libraries for this lab.

#### On POSIX systems (Linux, MacOSX, WSL, ...)

```bash
python3 -m venv env-lab4
source env-lab4/bin/activate
python3 -m pip install --upgrade pip
python3 -m pip install -r https://raw.githubusercontent.com/jeremiedecock/polytechnique-csc-52081-ep-2026-students/main/requirements-lab4.txt
```

#### On Windows

```bash
python3 -m venv env-lab4
env-lab4\Scripts\activate.bat
python3 -m pip install --upgrade pip
python3 -m pip install -r https://raw.githubusercontent.com/jeremiedecock/polytechnique-csc-52081-ep-2026-students/main/requirements-lab4.txt
```

### Import required packages

In [ ]:
import gymnasium as gym
import math
import numpy as np
import pandas as pd
from pathlib import Path

from typing import cast, List, Optional, Sequence, Tuple, Union

In [ ]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt

import seaborn as sns
from tqdm.notebook import tqdm

In [ ]:
from IPython.display import Video
from ipywidgets import interact

In [ ]:
import warnings

warnings.filterwarnings("ignore", category=UserWarning)

In [ ]:
sns.set_context("talk")

In [ ]:
FIGS_DIR = Path("figs/") / "lab4"       # Where to save figures (.gif or .mp4 files)
PLOTS_DIR = Path("figs/") / "lab4"      # Where to save plots (.png or .svg files)

In [ ]:
if not FIGS_DIR.exists():
    FIGS_DIR.mkdir(parents=True)
if not PLOTS_DIR.exists():
    PLOTS_DIR.mkdir(parents=True)

## Define the video selector widget

In [ ]:
def video_selector(file_path: List[Path]):
    return Video(file_path, embed=True, html_attributes="controls autoplay loop")

## Setup the FrozenLake toy problem with Gymnasium

For the purpose of focusing on the algorithms, we will use standard environments provided
by the Gymnasium framework. Especially, as in Lab 3, we will try to solve the FrozenLake-v1
problem (https://gymnasium.farama.org/environments/toy_text/frozen_lake/).
As a reminder, this environment is described [here](https://gymnasium.farama.org/environments/toy_text/frozen_lake/).

The action indices are outlined below:

| Action Index | Action     |
|--------------|------------|
| 0            | Move Left  |
| 1            | Move Down  |
| 2            | Move Right |
| 3            | Move Up    |

The following dictionary may be used to understand actions:

In [ ]:
action_labels = {0: "Move Left", 1: "Move Down", 2: "Move Right", 3: "Move Up"}

**Notice**: this environment is *fully observable*, thus here the terms (environment) *state* and (agent) *observation* are equivalent.
This is not always the case for example in poker, the agent doesn't know the opponent's cards.

### Display functions

The next cells contain three functions that can be used to display Q-tables, states and (greedy) policies in the FrozenLake environment.

In [ ]:
# Display the Q-table as a set of heatmaps, one for each action
def display_qtable(
    q_array: np.ndarray,
    title: str = "",
    figsize: Tuple[int, int] = (4, 4),
    annot: bool = True,
    fmt: str = "0.1f",
    linewidths: float = 0.5,
    square: bool = True,
    cbar: bool = False,
    cmap: str = "Reds",
    ticklabels: bool = False,
) -> None:
    """
    Display a Q-table as a set of heatmaps, one for each action.

    For the frozen lake environment, there are 16 states and 4 actions thus this function will display 4 heatmaps, one for each action.
    Each heatmap will display the Q-values for each state when performing the action indexed by the heatmap.

    Parameters
    ----------
    q_array : np.ndarray
        The Q-table to display. A 2D numpy array of 16x4 elements.
        Each row corresponds to a state, and each column corresponds to an action.
        In the frozen lake environment, there are 16 states and 4 actions thus the Q-table has a shape of (16, 4).
        For instance, q_array[0, 3] is the Q-value (estimation of the cumulative expected reward) for performing action 3 ("move up") in state 0 (the top left square).
    title : str, optional
        The title of the plot. Default is an empty string.
    figsize : tuple, optional
        The size of the figure (in inches), by default (4, 4)
    annot : bool, optional
        If True, write the data value in each cell, by default True
    fmt : str, optional
        The string formatting code to use when adding annotations, by default "0.1f" that will display a single decimal
    linewidths : float, optional
        The width of the lines that will divide each cell, by default .5
    square : bool, optional
        Whether to set the Axes aspect to "equal" so each cell is square-shaped, by default True
    cbar : bool, optional
        Whether to draw a colorbar, by default False
    cmap : str, optional
        The mapping from data values to color space, by default "Reds".
    ticklabels : bool, optional
        If True, display the tick labels. Default is False.

    Returns
    -------
    None
    """
    # Get the number of actions from the shape of the Q-table
    num_actions = q_array.shape[1]

    # Adjust the figure size (in inches) based on the number of actions
    global_figsize = list(figsize)
    global_figsize[0] *= num_actions

    # Create a subplot for each action
    fig, ax_list = plt.subplots(ncols=num_actions, figsize=global_figsize)

    # For each action, display the Q-values for all states as a heatmap
    for action_index in range(num_actions):
        ax = ax_list[action_index]

        # Retrieve the Q-values for each state when performing the action indexed by "action_index".
        # This forms a 1D array, state_vec, where state_vec[i] = Q(i, action_index).
        state_vec = q_array[:, action_index]

        # Display the Q-values for each state when performing the action indexed by "action_index"
        # i.e. display Q(., action_index)
        display_state(
            state_vec,
            title=r"$Q(\cdot,a_{})$".format(action_index),
            # title=r"$Q(\cdot,a_{})$ {}".format(action_index, action_labels[action_index]),
            figsize=figsize,
            annot=annot,
            fmt=fmt,
            linewidths=linewidths,
            square=square,
            cbar=cbar,
            cmap=cmap,
            ticklabels=ticklabels,
            ax=ax,
        )

    # Set the title for the entire figure
    plt.suptitle(title)
    # Display the figure
    plt.show()

In [ ]:
def display_state(
    state_seq: Union[List[int], List[float], np.ndarray],
    title: str = "",
    figsize: Tuple[int, int] = (5, 5),
    annot: bool = True,
    fmt: str = "0.1f",
    linewidths: float = 0.5,
    square: bool = True,
    cbar: bool = False,
    cmap: str = "Reds",
    ticklabels: bool = False,
    ax: Optional[matplotlib.axes.Axes] = None,
) -> Union[matplotlib.axes.Axes, None]:
    """
    Display the expected values of all states as a heatmap.

    Parameters
    ----------
    state_seq : list of int, list of float or 1D numpy array of 16 elements
        The sequence of expected values to display. This can be a list, a 1D array, etc.
        Each element is the estimation of the expected value of the corresponding state.
        For example, state_seq[0] is the estimation of the expected value of the first state.
        There are 16 elements in this sequence for the frozenlake environment, i.e., one per state of the environment.
    title : str, optional
        The title of the plot, by default an empty string
    figsize : tuple, optional
        The size of the figure (in inches), by default (5, 5)
    annot : bool, optional
        If True, write the data value in each cell, by default True
    fmt : str, optional
        The string formatting code to use when adding annotations, by default "0.1f"
    linewidths : float, optional
        The width of the lines that will divide each cell, by default .5
    square : bool, optional
        Whether to set the Axes aspect to "equal" so each cell is square-shaped, by default True
    cbar : bool, optional
        Whether to draw a colorbar, by default False
    cmap : str, optional
        The mapping from data values to color space, by default "Reds".
    ticklabels : bool, optional
        If True, display the tick labels. Default is False.
    ax : matplotlib.axes.Axes, optional
        The axes object to draw the heatmap on, by default None

    Returns
    -------
    matplotlib.axes.Axes, optional
        The axes object with the heatmap if one was provided, otherwise None.
    """
    # Calculate the size of the state array
    size = int(math.sqrt(len(state_seq)))

    # Convert the state sequence to a numpy array (if it isn't already one)
    state_array = np.array(state_seq)

    # Reshape the state array into a square matrix
    # (we assume here that the original frozen lake environment is used,
    # thus the state space can be visualized as a square grid)
    state_array = state_array.reshape(size, size)

    # If no axes object is provided, create a new figure and axes
    if ax is None:
        fig, ax = plt.subplots(figsize=figsize)

    # Create a heatmap of the state array on the axes
    ax = sns.heatmap(
        state_array,
        annot=annot,
        fmt=fmt,
        linewidths=linewidths,
        square=square,
        cbar=cbar,
        cmap=cmap,
        xticklabels=ticklabels,
        yticklabels=ticklabels,
        ax=ax,
    )

    # If a title is provided, set the title of the plot
    if title != "":
        if ax is None:
            plt.title(title)
        else:
            ax.set_title(title)

    # If no axes object was provided, display the plot
    # Otherwise, return the axes object with the heatmap
    if ax is None:
        plt.show()
        return None
    else:
        return ax

In [ ]:
def display_policy(policy: List[int], environment: gym.Env) -> None:
    """
    Display the policy as a heatmap.

    Parameters
    ----------
    policy : list of int
        The policy to be displayed. Each integer represents an action to be taken in a state.

    Returns
    -------
    None
    """
    # Create a list of actions with their corresponding labels
    actions_src = [
        f"{action}={action_labels[action].replace('Move ', '')}" for action in range(environment.action_space.n)
    ]

    # Create a title for the heatmap using the actions and their labels
    title = f"Policy ({', '.join(actions_src)})"

    # Use the display_state function to create a heatmap of the policy
    # The fmt parameter is set to "d" to display integers, cbar is set to False to not display a colorbar,
    # and cmap is set to "Reds" to use the Reds color map
    display_state(policy, title=title, fmt="d", cbar=False, cmap="Reds")

## Notation Table (common to TD Learning, SARSA, and Q-Learning)

| Symbol                           | Meaning                                                                            |
| -------------------------------- | ---------------------------------------------------------------------------------- |
| $\mathcal{S}$                    | Set of all states of the MDP                                                       |
| $\mathcal{S}^F$                  | Set of final (terminal) states                                                     |
| $\mathcal{S}^+$                  | Set of all non-final states, i.e. $\mathcal{S} \setminus \mathcal{S}^F$            |
| $\mathcal{A}(s)$                 | Set of actions available in state $s$                                              |
| $s, s'$                          | Current state and next state after a transition                                    |
| $s_F$                            | A final (terminal) state                                                           |
| $a, a'$                          | Current action and next action                                                     |
| $r$                              | Immediate reward observed after taking an action                                   |
| $\pi(s)$                         | Policy: mapping from a state to an action                                          |
| $\epsilon$                       | Exploration parameter for $\epsilon$-greedy action selection                       |
| $\gamma \in [0,1]$               | Discount factor                                                                    |
| $\alpha \in (0,1]$               | Learning rate (step size)                                                          |
| $V(s)$                           | Estimated value of state $s$ under policy $\pi$                                    |
| $Q(s,a)$                         | Estimated action-value of taking action $a$ in state $s$                           |


## Exercise 1: TD Learning

**Notice**: Here we assume that the reward only depends on the state: $r(\boldsymbol{s}) \equiv r(\boldsymbol{s}, \boldsymbol{a}, \boldsymbol{s}')$.

**Notice**:
-  $\mathcal{S}$ is the set of all nonterminal states
-  $\mathcal{S^F}$ is the set of all terminal states
-  $\mathcal{S^+}$ is the set of all states, including the terminal states

In Lab3, we explored Dynamic Programming methods that can be used to solve a Markov Decision Process (MDP) when the environment is perfectly known to the agent, i.e., in cases where the agent knows the transition and reward functions in advance. However, this is a strong assumption, as in most practical problems, these functions are not known beforehand. In this lab, we will learn how to create agents that can solve Markov Decision Problems without prior knowledge of the environment. These agents learn the dynamics of their environment by exploring it and use this knowledge to find an optimal policy.

We will start with the *TD Learning* (*Temporal Difference Learning*) algorithm, which can be used to **evaluate** any **given policy** (i.e. compute the *value function* of the environment following the given policy).
Variants of TD Learning can also be adapted for optimal control, such as in applications like [TD-Gammon](https://en.wikipedia.org/wiki/TD-Gammon).
Exercises 2 and 3 also reuse the main concepts of TD Learning to calculate an optimal policy.

The algorithm is outlined below.

---

### The TD Learning algorithm

<b>Input</b>:<br>
	$\quad\quad$ the policy $\pi$ to be evaluated<br>
<b>Algorithm parameter</b>:<br>
	$\quad\quad$ discount factor $\gamma$<br>
	$\quad\quad$ step size $\alpha \in ]0,1]$<br><br>

Initialize $V(\boldsymbol{s})$ arbitrarily $\forall \boldsymbol{s} \in \mathcal{S^+}$, except that $V(\boldsymbol{s}_F) \leftarrow 0 ~ \forall \boldsymbol{s}_F \in \mathcal{S^F}$ (final states)<br>

<b>FOR EACH</b> episode<br>
	$\quad$ $\boldsymbol{s} \leftarrow \text{env.reset}() \quad\quad\quad\quad\quad\quad~~$ <span style="color: gray;">👈 initialize ***s***</span><br>
	$\quad$ <b>DO</b> <br>
		$\quad\quad$ $\boldsymbol{a} \leftarrow \pi(\boldsymbol{s})$<br>
		$\quad\quad$ $\boldsymbol{s}', r \leftarrow \text{env.step}(\boldsymbol{a}) \quad\quad\quad\quad$ <span style="color: gray;">👈 take action ***a***, observe *r* and ***s'***</span><br>
		$\quad\quad$ $V(\boldsymbol{s}) \leftarrow V(\boldsymbol{s}) + \alpha \left[ \underbrace{\overbrace{r + \gamma ~ V(\boldsymbol{s}')}^{\text{Target for } V(\boldsymbol{s})} ~ - ~ V(\boldsymbol{s})}_{\text{TD error}} \right]$<br>
		$\quad\quad$ $\boldsymbol{s} \leftarrow \boldsymbol{s}'$<br>
	$\quad$ <b>UNTIL</b> $\boldsymbol{s}$ is final

**Notice**: in the following cell, `policy` is a list of actions (one per state see two cells bellow).

### Task 1: Implement the TD Learning algorithm

In [ ]:
DISPLAY_EVERY_N_EPISODES = 50

def td_learning(
    policy: Sequence[int],
    environment: gym.Env,
    alpha: float = 0.1,
    alpha_factor: float = 0.995,
    gamma: float = 0.95,
    num_episodes: int = 1000,
    display: bool = False,
) -> Tuple[np.ndarray, List[np.ndarray], List[float]]:
    """
    Perform Temporal Difference learning on a given policy and environment.

    Parameters
    ----------
    policy : Sequence[int]
        The policy to be learned, represented as a sequence mapping states (the index of the sequence) to actions (the value of the sequence for this index).
        For example, policy[0] is the action to take in state 0.
    environment : gym.Env
        The environment in which the agent operates.
    alpha : float, optional
        The learning rate, between 0 and 1. By default 0.1
    alpha_factor : float, optional
        The factor by which the learning rate alpha decreases each episode, by default 0.995
    gamma : float, optional
        The discount factor, between 0 and 1. By default 0.95
    num_episodes : int, optional
        The number of episodes to run, by default 1000
    display : bool, optional
        Whether to display the value function (every DISPLAY_EVERY_N_EPISODES episodes), by default False

    Returns
    -------
    Tuple[np.ndarray, List[np.ndarray], List[float]]
        A tuple containing:
        - The learned value function (1D ndarray of shape (16,))
        - The history of value functions over episodes
        - The history of alpha values over episodes
    """
    # Initialize the history of the value function and the learning rate
    v_array_history: List[np.ndarray] = []
    alpha_history: List[float] = []

    observation_space = cast(gym.spaces.Discrete, environment.observation_space)

    # Get the number of states in the environment
    num_states = observation_space.n

    # Initialize the value function to zeros
    v_array = np.zeros(num_states)

    # Loop over the episodes
    for episode_index in tqdm(range(1, num_episodes)):
        # Display the value function every DISPLAY_EVERY_N_EPISODES episodes if display is True
        if display and episode_index % DISPLAY_EVERY_N_EPISODES == 0:
            display_state(
                v_array,
                title=f"Value function (ep. {episode_index})",
                cbar=True,
                cmap="Reds",
            )

        # Save the current value function and learning rate
        v_array_history.append(v_array.copy())
        alpha_history.append(alpha)

        # Decrease the learning rate if alpha_factor is not None
        if alpha_factor is not None:
            alpha = alpha * alpha_factor

        # TODO 👈 Complete the implementation of the Temporal-Difference (TD) algorithm

    # Return the learned value function
    return v_array, v_array_history, alpha_history

**Note**: In the following cell, the `display` argument can be set to `True` to see the evolution of the value function `v_array` over iterations.

In [ ]:
# Define the policy to evaluate
policy = [0, 3, 3, 3,
          0, 0, 0, 0,
          3, 1, 0, 0,
          0, 2, 1, 0]

# Create the environment and set the maximum number of steps per episode
environment = gym.make("FrozenLake-v1", max_episode_steps=1000)

# Apply Temporal Difference (TD) Learning to calculate the value function for the policy defined earlier, within the context of the FrozenLake environment.
v_array_ex1, v_array_history_ex1, alpha_history_ex1 = td_learning(policy, environment, display=False)

environment.close()

In [ ]:
# Display the learned value function
display_state(v_array_ex1, title="Value function", cbar=True, cmap="Reds");

### Display the evolution of the action-value function over iterations

In [ ]:
# Print the evolution of the value function during the learning process
df_v_hist_ex1 = pd.DataFrame(v_array_history_ex1)
df_v_hist_ex1

Evolution of `v_array_ex1` (the estimated expected value of each state) over iterations (one curve per state):

In [ ]:
# Show the evolution of the estimated expected value for each state during the learning process
df_v_hist_ex1.plot(figsize=(14, 8))
plt.title(r"$V^{\pi}(\cdot)$ w.r.t iteration")
plt.ylabel(r"$V^{\pi}(\cdot)$")
plt.xlabel("iteration")
plt.legend(loc="upper right");

In [ ]:
# Show the evolution of the alpha parameter during the learning process
plt.loglog(alpha_history_ex1)
plt.title("Alpha w.r.t iteration")
plt.ylabel("Alpha")
plt.xlabel("iteration");

### Question 1

Copy and paste the output of the following cell into the first question of the [Lab 4 - Evaluation](https://moodle.ip-paris.fr/mod/quiz/view.php?id=185078) in Moodle:  
*"What is the expected value of the 9th state (i.e. `v_array[8]`) in the FrozenLake environment for the given policy π using the default parameters?"*

In [ ]:
# %%time

NUM_RUNS = 10

# Define the policy to evaluate
policy = [0, 3, 3, 3,
          0, 0, 0, 0,
          3, 1, 0, 0,
          0, 2, 1, 0]

# Create the environment and set the maximum number of steps per episode
environment = gym.make("FrozenLake-v1", max_episode_steps=1000)

run_list = []
for _ in tqdm(range(1, NUM_RUNS)):
    v_array, _, _ = td_learning(policy, environment, display=False)
    run_list.append(v_array[8].item())

environment.close()

print(f"\n\n⚠️ Copy and paste this value into Moodle 👉 {np.mean(run_list).item():.2f}\n(it has the required number of decimals)\n\n")

In [ ]:
# print(np.std(run_list))
# pd.DataFrame(run_list).plot.hist();

### Task 2: The learning rate $\alpha$ in TD-Learning

In the previous exercise, set `alpha_factor` to 1 then check the algorithm with different `alpha` values between 0.1 and 1.

In [ ]:
# Define the policy to evaluate
policy = [0, 3, 3, 3,
          0, 0, 0, 0,
          3, 1, 0, 0,
          0, 2, 1, 0]

# Create the environment and set the maximum number of steps per episode
environment = gym.make("FrozenLake-v1", max_episode_steps=1000)

# Apply Temporal Difference (TD) Learning to calculate the value function for the policy defined earlier, within the context of the FrozenLake environment.
_, v_array_history, _ = td_learning(policy, environment, display=False, alpha_factor=1, alpha=0.5)  # 👈 Test different `alpha` values here

df = pd.DataFrame(v_array_history)

# Show the evolution of the estimated expected value for each state during the learning process
df.plot(figsize=(14, 8))
plt.title(r"$V^{\pi}(\cdot)$ w.r.t iteration")
plt.ylabel(r"$V^{\pi}(\cdot)$")
plt.xlabel("iteration")
plt.legend(loc="upper right");

environment.close()

### Bonus question

What do you observe when you change the `alpha_factor` ?
What is the role of this parameter ?

## Exercise 2: SARSA

### The Greedy and Epsilon Greedy policies

In exercise 1, **TD-Learning** has been used to **estimate the value function** for a **given policy**.  
In the following exercises, we will now explore how to **find the optimal** (or a nearly optimal) policy.  
To do that, we will use two algorithms (*SARSA* and *Q-Learning*) that estimate a *Q-table* instead of a *V-table*.  

This *Q-table* provides the expected **cumulative reward** when the agent takes a given action $\boldsymbol{a}$ from any state $\boldsymbol{s}$ and then follows a specified *behavior policy* to choose subsequent actions until a terminal state is reached. (This behavior policy may also serve as an *exploration policy* guiding action selection.)  

While exploring the environment, the agent **updates** its *Q-table* using a **rule derived from the Bellman equation** (i.e., an *update rule*).  

The purpose of this exercise is to implement the *greedy* and the $\epsilon$-*greedy* policies that agents will use to explore the environment and update their Q-Table:

$\displaystyle \pi^{Q^{\pi}}(\boldsymbol{s}) := \text{greedy}(\boldsymbol{s}, Q^{\pi}) = \arg\max_{\boldsymbol{a} \in \mathcal{A}} Q^{\pi}(\boldsymbol{s}, \boldsymbol{a})$


$\pi^{Q^{\pi}}_{\epsilon}(\boldsymbol{s}) := \epsilon\text{-greedy}(\boldsymbol{s}, Q^{\pi}) = $
randomly choose between $\underbrace{\text{greedy}(\boldsymbol{s}, Q^{\pi})}_{\text{with probability } 1 - \epsilon}$
and $~~ \underbrace{\text{a random action}}_{\text{with probability } \epsilon}$    i.e. $\epsilon \in (0,1]$

### Task 1: Implement the Greedy and Epsilon Greedy policies

In [ ]:
def greedy_policy(state: int, q_array: np.ndarray) -> int:
    """
    Determine the action that maximizes the Q-value for a given state.

    Parameters
    ----------
    state : int
        The current state.
    q_array : np.ndarray
        The Q-table.

    Returns
    -------
    int
        The action that maximizes the Q-value for the given state.
    """
    # Find the action that maximizes the Q-value for the given state
    action = ... # TODO 👈 Complete the implementation of the greedy policy

    return action


def epsilon_greedy_policy(state: int, q_array: np.ndarray, epsilon: float) -> int:
    """
    Determine the action to take based on an epsilon-greedy policy.

    Parameters
    ----------
    state : int
        The current state.
    q_array : np.ndarray
        The Q-table.
    epsilon : float
        The probability of choosing a random action.

    Returns
    -------
    int
        The action to take.
    """
    # TODO 👈 Complete the implementation of the epsilon greedy policy

    return action

### The SARSA algorithm

To find the optimal policy (or a nearly optimal policy) for the FrozenLake-v1 problem, we will first use the SARSA algorithm.
It is based on the online update of the *QTable* for the current policy, defined as:
$$
Q(s, a) = \mathbb{E}^{\pi} \left[ \sum_{t=0}^{H} \gamma^t r_{t}(s_t, a_t) | s_t=s_0, a=a_0 \right] ,
$$
where $\gamma \in (0, 1]$ is the discount factor, and $H$ the horizon of the episode.

The SARSA algorithm updates a tabular estimate of the Q-function using the following update rule:
$$
Q(s, a) \leftarrow Q(s, a) +
\alpha
\left[
\underbrace{\overbrace{r + \gamma Q(s', a')}^{\text{Target}} - Q(s, a)}_{\text{TD error}}
\right] ,
$$
where $\alpha \in (0, 1]$ is the learning rate, and $r_t$ is the reward received by the agent at time step $t$.
Most of the time, the SARSA algorithm is implemented with an $\epsilon$-greedy exploration strategy.
This strategy consists in selecting the best action learned so far with probability $(1 - \epsilon)$ and to select a random
action with probability $\epsilon$.

---

***The SARSA algorithm***

<b>Input</b>:<br>
	$\quad\quad$ none<br>
<b>Algorithm parameter</b>:<br>
	$\quad\quad$ discount factor $\gamma$<br>
	$\quad\quad$ step size $\alpha \in ]0,1]$<br>
	$\quad\quad$ small $\epsilon > 0$<br><br>

Initialize $Q(\boldsymbol{s}, \boldsymbol{a})$ arbitrarily $\forall \boldsymbol{s} \in \mathcal{S}, \boldsymbol{a} \in \mathcal{A}(\boldsymbol{s})$,
except that $Q(\boldsymbol{s}_F, \cdot) \leftarrow 0 ~~~ \forall \boldsymbol{s}_F \in \mathcal{S}^F$ (final states)<br><br>

<b>FOR EACH</b> episode<br>
	$\quad$ $\boldsymbol{s} \leftarrow \text{env.reset}() \quad\quad\quad\quad\quad\quad~~$ <span style="color: gray;">👈 initialize ***s***</span><br>
	$\quad$ $\boldsymbol{a} \leftarrow \epsilon\text{-greedy}(\boldsymbol{s}, Q)$<br>
	$\quad$ <b>DO</b> <br>
		$\quad\quad$ $r, \boldsymbol{s}' \leftarrow \text{env.step}(\boldsymbol{a}) \quad\quad\quad\quad$ <span style="color: gray;">👈 take action ***a***, observe *r* and ***s'***</span><br>
		$\quad\quad$ $\boldsymbol{a}' \leftarrow \epsilon\text{-greedy}(\boldsymbol{s}', Q)$<br>
		$\quad\quad$ $Q(\boldsymbol{s},\boldsymbol{a}) \leftarrow Q(\boldsymbol{s},\boldsymbol{a}) + \alpha \left[ \underbrace{\overbrace{r + \gamma Q(s', a')}^{\text{Target}} ~ - ~ Q(\boldsymbol{s},\boldsymbol{a})}_{\text{TD error}} \right]$<br>
		$\quad\quad$ $\boldsymbol{s} \leftarrow \boldsymbol{s}'$<br>
		$\quad\quad$ $\boldsymbol{a} \leftarrow \boldsymbol{a}'$<br>
	$\quad$ <b>UNTIL</b> $\boldsymbol{s}$ is final

### Task 2: Implement the SARSA algorithm

**Tasks**: Implement the SARSA algorithm with $\epsilon$-greedy exploration (start with $\epsilon = 0.5$).

In [ ]:
DISPLAY_EVERY_N_EPISODES = 50

def sarsa(
    environment: gym.Env,
    alpha: float = 0.1,
    alpha_factor: float = 0.9995,
    gamma: float = 0.99,
    epsilon: float = 0.5,
    num_episodes: int = 10000,
    display: bool = False,
) -> Tuple[np.ndarray, List[np.ndarray], List[float]]:
    """
    Learn an action-value function Q(s, a) with the on-policy SARSA algorithm
    using an epsilon-greedy behavior policy.

    The update performed at each step is:
        Q(s, a) <- Q(s, a) + alpha * (r + gamma * Q(s', a') - Q(s, a))

    Parameters
    ----------
    environment : gym.Env
        Environment following the (Gymnasium) API:
        - reset() -> (observation, info)
        - step(action) -> (observation, reward, terminated, truncated, info)
        The observation space and action space are assumed to be discrete.
    alpha : float, optional
        Initial learning rate in (0, 1]. Default is 0.1.
    alpha_factor : float, optional
        Multiplicative decay applied to `alpha` after each episode. If None,
        `alpha` stays constant. Default is 0.9995.
    gamma : float, optional
        Discount factor in (0, 1]. Default is 0.99.
    epsilon : float, optional
        Epsilon for epsilon-greedy action selection (probability of random
        action). Default is 0.5.
    num_episodes : int, optional
        Number of training episodes. Default is 10000.
    display : bool, optional
        If True, display the Q-table every `DISPLAY_EVERY_N_EPISODES` episodes.
        Default is False.

    Returns
    -------
    q_array : np.ndarray
        Learned Q-table of shape (num_states, num_actions).
        For instance, for the FrozenLake environment, q_array[0, 3] is
        the Q-value (estimation of the cumulative expected reward) for performing
        action 3 ("move up") in state 0 (the top left square).
    q_array_history : List[np.ndarray]
        Snapshot of the Q-table taken once per episode (before training within
        that episode).
    alpha_history : List[float]
        Value of `alpha` used for each episode (recorded before applying decay).
    """
    # Initialize the history of the Q-table and learning rate
    q_array_history = []
    alpha_history = []

    observation_space = cast(gym.spaces.Discrete, environment.observation_space)
    action_space = cast(gym.spaces.Discrete, environment.action_space)

    # Get the number of states in the environment
    num_states = observation_space.n

    # Get the number of actions in the environment
    num_actions = action_space.n

    # Initialize the Q-table to zeros
    q_array = np.zeros([num_states, num_actions])

    # Loop over the episodes
    for episode_index in tqdm(range(1, num_episodes)):
        # Display the Q-table every DISPLAY_EVERY_N_EPISODES episodes if display is True
        if display and episode_index % DISPLAY_EVERY_N_EPISODES == 0:
            display_qtable(q_array, title="Q table")

        # Save the current Q-table and learning rate
        q_array_history.append(q_array.copy())
        alpha_history.append(alpha)

        # Decrease the learning rate if alpha_factor is not None
        if alpha_factor is not None:
            alpha = alpha * alpha_factor

        # TODO 👈 Complete the implementation of the SARSA algorithm

    # Return the learned Q-table
    return q_array, q_array_history, alpha_history

**Note**: In the following cell, the `display` argument can be set to `True` to see the evolution of the action-value function `q_array` over iterations.

In [ ]:
# Create the environment and set the maximum number of steps per episode
environment = gym.make("FrozenLake-v1", max_episode_steps=1000)

# Apply SARSA to calculate the Q-table for the FrozenLake environment
q_array_ex2, q_array_history_ex2, alpha_history_ex2 = sarsa(environment, display=False)

environment.close()

In [ ]:
# Display the learned Q-table
display_qtable(q_array_ex2, title="Q Table")

### Display the greedy policy

In [ ]:
policy_ex2 = [greedy_policy(state, q_array_ex2) for state in range(environment.observation_space.n)]
policy_ex2

In [ ]:
display_policy(policy_ex2, environment)

### Display the evolution of the action-value function over iterations

In [ ]:
# Make a Pandas dataframe containing the evolution of the Q-table during the learning process
q_array_history_array_ex2 = np.array(q_array_history_ex2)
df_q_hist_list_ex2 = []

for action_index in range(q_array_history_array_ex2.shape[2]):
    df_q_hist_list_ex2.append(pd.DataFrame(q_array_history_array_ex2[:, :, action_index]))

Evolution of `q_array_ex2` over iterations (one curve per state):

In [ ]:
# Show the evolution of the estimated expected value for each action and state during the learning process
for action_index, df_q_hist in enumerate(df_q_hist_list_ex2):
    df_q_hist.plot(figsize=(14,8))
    plt.title(r'$Q(\cdot,a_{})$ w.r.t iteration with $a_{}$ := "{}"'.format(action_index, action_index, action_labels[action_index]))
    plt.ylabel(r"$Q(\cdot,a_{})$".format(action_index))
    plt.xlabel("iteration")
    plt.legend(loc="upper right")

In [ ]:
# Show the evolution of the alpha parameter during the learning process
plt.loglog(alpha_history_ex2)
plt.title("Alpha w.r.t iteration")
plt.ylabel("Alpha")
plt.xlabel("iteration")

### Evaluate Policy with Gymnasium

As a measure of performance, count the number of successful trials on 1000 episodes.

**Note**: Gymnasium considers the task is solved if you reach 76\% of success over the episodes.

In [ ]:
environment = gym.make("FrozenLake-v1", max_episode_steps=1000)

reward_list = []
NUM_EPISODES = 1000

for episode_index in range(NUM_EPISODES):
    state, info = environment.reset()
    episode_over = False

    while not episode_over:
        # action = epsilon_greedy_policy(state, q_array_ex2, epsilon)
        action = greedy_policy(state, q_array_ex2)
        state, reward, terminated, truncated, info = environment.step(action)
        episode_over = terminated or truncated

    reward_list.append(reward)

reward_df = pd.DataFrame(reward_list)

print('Average reward (which is equivalent to a "success rate" in the FrozenLake environment as the total rewards in this environment are either 0 or 1):', np.average(reward_df))

environment.close()

### Test the optimized policy

In [ ]:
VIDEO_PREFIX_EX2 = "lab4_ex2_sarsa"

NUM_EPISODES = 3

file_path_list = [
    FIGS_DIR / f"{VIDEO_PREFIX_EX2}-episode-{episode_index}.mp4"
    for episode_index in range(NUM_EPISODES)
]

for file_path in file_path_list:
    file_path.unlink(missing_ok=True)

environment = gym.make("FrozenLake-v1", max_episode_steps=1000, render_mode="rgb_array")
environment = gym.wrappers.RecordVideo(
    environment,
    video_folder=str(FIGS_DIR),
    name_prefix=VIDEO_PREFIX_EX2,
    episode_trigger=lambda x: True
)
environment = gym.wrappers.RecordEpisodeStatistics(environment, buffer_length=NUM_EPISODES)

for episode_index in range(NUM_EPISODES):
    state, info = environment.reset()

    episode_over = False
    while not episode_over:
        # action = epsilon_greedy_policy(state, q_array_ex2, epsilon)
        action = greedy_policy(state, q_array_ex2)
        state, reward, terminated, truncated, info = environment.step(action)
        episode_over = terminated or truncated

    reward_list.append(reward)

print(f'Episode time taken: {environment.time_queue}')
print(f'Episode total rewards: {environment.return_queue}')
print(f'Episode lengths: {environment.length_queue}')

environment.close()

print("\nSelect the episode to play here 👇\n")

interact(video_selector, file_path=file_path_list);

### Question 2

Copy and paste the output of the following cell into the second question of the [Lab 4 - Evaluation](https://moodle.ip-paris.fr/mod/quiz/view.php?id=185078) in Moodle:
*"What is the Q-value Q(s0,a0) of the initial state and action 0 ("Move Left") in the FrozenLake environment using the default parameters?"*

In [ ]:
# %%time

NUM_RUNS = 10

# Create the environment and set the maximum number of steps per episode
environment = gym.make("FrozenLake-v1", max_episode_steps=1000)

run_list = []
for _ in tqdm(range(1, NUM_RUNS)):
    q_array, _, _ = sarsa(environment, display=False)
    run_list.append(q_array[0][0].item())

environment.close()

print(f"\n\n⚠️ Copy and paste this value into Moodle 👉 {np.mean(run_list).item():.2f}\n(it has the required number of decimals)\n\n")

In [ ]:
# print(np.std(run_list))
# pd.DataFrame(run_list).plot.hist();

## Exercise 3: QLearning

Another reinforcement learning algorithm is *Q-Learning*.
The fundamental difference from *SARSA* is that it is an *off-policy* algorithm.
This means it learns from trajectories generated by a policy different from the one it is optimizing.
In other words, the *behavior policy* (*epsilon-greedy*) is different from the *target policy* (*greedy*).

To do so, it uses the following update rule:
$$
Q(s, a) \leftarrow Q(s, a) +
\alpha
\left[
\underbrace{\overbrace{r + \gamma \max_{a^* \in \mathcal{A}} Q(s', a^*)}^{\text{Target}} - Q(s, a)}_{\text{TD error}}
\right] ,
$$

**Task**: in this exercise, you will replace the SARSA update rule by the Q-learning one and analyze the
differences in performance.

---

### The QLearning algorithm

<b>Input</b>:<br>
	$\quad\quad$ none<br>
<b>Algorithm parameter</b>:<br>
	$\quad\quad$ discount factor $\gamma$<br>
	$\quad\quad$ step size $\alpha \in ]0,1]$<br>
	$\quad\quad$ small $\epsilon > 0$<br><br>

Initialize $Q(\boldsymbol{s}, \boldsymbol{a})$ arbitrarily $\forall \boldsymbol{s} \in \mathcal{S}, \boldsymbol{a} \in \mathcal{A}(\boldsymbol{s})$,
except that $Q(\boldsymbol{s}_F, \cdot) \leftarrow 0 ~~~ \forall \boldsymbol{s}_F \in \mathcal{S}^F$ (final states)<br><br>

<b>FOR EACH</b> episode<br>
	$\quad$ $\boldsymbol{s} \leftarrow \text{env.reset}() \quad\quad\quad\quad\quad\quad~~$ <span style="color: gray;">👈 initialize ***s***</span><br>
	$\quad$ <b>DO</b> <br>
		$\quad\quad$ $\boldsymbol{a} \leftarrow \epsilon\text{-greedy}(\boldsymbol{s}, Q)$<br>
		$\quad\quad$ $r, \boldsymbol{s}' \leftarrow \text{env.step}(\boldsymbol{a}) \quad\quad\quad\quad$ <span style="color: gray;">👈 take action ***a***, observe *r* and ***s'***</span><br>
		$\quad\quad$ $Q(\boldsymbol{s},\boldsymbol{a}) \leftarrow Q(\boldsymbol{s},\boldsymbol{a}) + \alpha \left[ \underbrace{\overbrace{r + \gamma ~ \max_{\boldsymbol{a^*} \in \mathcal{A}} Q(\boldsymbol{s}', \boldsymbol{a^*})}^{\text{Target}} ~ - ~ Q(\boldsymbol{s},\boldsymbol{a})}_{\text{TD error}} \right]$<br>
		$\quad\quad$ $\boldsymbol{s} \leftarrow \boldsymbol{s}'$<br>
	$\quad$ <b>UNTIL</b> $\boldsymbol{s}$ is final

### Task 1: Implement the QLearning algorithm

In [ ]:
DISPLAY_EVERY_N_EPISODES = 50

def q_learning(
    environment: gym.Env,
    alpha: float = 0.1,
    alpha_factor: float = 0.9995,
    gamma: float = 0.99,
    epsilon: float = 0.5,
    num_episodes: int = 10000,
    display: bool = False,
) -> Tuple[np.ndarray, List[np.ndarray], List[float]]:
    """
    Train an agent with the tabular Q-learning algorithm (off-policy TD control).

    The agent follows an epsilon-greedy *behavior* policy derived from the current Q-table,
    while the update uses the greedy action in the next state (the *target* policy):

        Q[s, a] <- Q[s, a] + alpha * (r + gamma * max_a' Q[s', a'] - Q[s, a])

    Parameters
    ----------
    environment : gym.Env
        Gymnasium environment with discrete observation and action spaces
        (i.e., ``environment.observation_space.n`` and ``environment.action_space.n`` exist).
    alpha : float, optional
        Initial learning rate (step size) in (0, 1]. Default is 0.1.
    alpha_factor : float, optional
        Multiplicative decay applied to ``alpha`` after each episode. If None, no decay is
        applied. Default is 0.9995.
    gamma : float, optional
        Discount factor in [0, 1]. Default is 0.99.
    epsilon : float, optional
        Exploration probability for the epsilon-greedy behavior policy. Default is 0.5.
    num_episodes : int, optional
        Number of training episodes. Default is 10000.
    display : bool, optional
        If True, periodically visualizes the current Q-table every
        ``DISPLAY_EVERY_N_EPISODES`` episodes. Default is False.

    Returns
    -------
    q_array : np.ndarray
        Learned Q-table of shape (num_states, num_actions).
        For instance, for the FrozenLake environment, q_array[0, 3] is
        the Q-value (estimation of the cumulative expected reward) for performing
        action 3 ("move up") in state 0 (the top left square).
    q_array_history : List[np.ndarray]
        Snapshot of the Q-table taken once per episode (before the episode starts).
        Length is ``num_episodes - 1`` due to the episode loop using ``range(1, num_episodes)``.
    alpha_history : List[float]
        Value of ``alpha`` recorded once per episode (before applying ``alpha_factor``).
        Length matches ``q_array_history``.

    Notes
    -----
    This implementation assumes the environment follows the Gymnasium step API:
    ``new_state, reward, terminated, truncated, info = env.step(action)``.
    """
    # Initialize the history of the Q-table and learning rate
    q_array_history = []
    alpha_history = []

    observation_space = cast(gym.spaces.Discrete, environment.observation_space)
    action_space = cast(gym.spaces.Discrete, environment.action_space)

    # Get the number of states in the environment
    num_states = observation_space.n

    # Get the number of actions in the environment
    num_actions = action_space.n

    # Initialize the Q-table to zeros
    q_array = np.zeros([num_states, num_actions])

    # Loop over the episodes
    for episode_index in tqdm(range(1, num_episodes)):
        # Display the Q-table every DISPLAY_EVERY_N_EPISODES episodes if display is True
        if display and episode_index % DISPLAY_EVERY_N_EPISODES == 0:
            display_qtable(q_array, title="Q table")

        # Save the current Q-table and learning rate
        q_array_history.append(q_array.copy())
        alpha_history.append(alpha)

        # Decrease the learning rate if alpha_factor is not None
        if alpha_factor is not None:
            alpha = alpha * alpha_factor

        # TODO 👈 Complete the implementation of the Q-learning algorithm

    # Return the learned Q-table
    return q_array, q_array_history, alpha_history

**Note**: In the following cell, the `display` argument can be set to `True` to see the evolution of the action-value function `q_array` over iterations.

In [ ]:
# Create the environment and set the maximum number of steps per episode
environment = gym.make("FrozenLake-v1", max_episode_steps=1000)

# Apply Q-learning to calculate the Q-table for the FrozenLake environment
q_array_ex3, q_array_history_ex3, alpha_history_ex3 = q_learning(environment, display=False)

environment.close()

In [ ]:
# Display the learned Q-table
display_qtable(q_array_ex3, title="Q Table")

### Display the greedy policy

In [ ]:
policy_ex3 = [greedy_policy(state, q_array_ex3) for state in range(environment.observation_space.n)]
policy_ex3

In [ ]:
display_policy(policy_ex3, environment)

### Display the evolution of the value function over iterations

In [ ]:
# Make a Pandas dataframe containing the evolution of the Q-table during the learning process
q_array_history_array_ex3 = np.array(q_array_history_ex3)
df_q_hist_list_ex3 = []

for action_index in range(q_array_history_array_ex3.shape[2]):
    df_q_hist_list_ex3.append(pd.DataFrame(q_array_history_array_ex3[:, :, action_index]))

Evolution of `q_array_ex3` over iterations (one curve per state):

In [ ]:
# Show the evolution of the estimated expected value for each action and state during the learning process
for action_index, df_q_hist in enumerate(df_q_hist_list_ex3):
    df_q_hist.plot(figsize=(14,8))
    plt.title(r'$Q(\cdot,a_{})$ w.r.t iteration with $a_{}$ := "{}"'.format(action_index, action_index, action_labels[action_index]))
    plt.ylabel(r"$Q(\cdot,a_{})$".format(action_index))
    plt.xlabel("iteration")
    plt.legend(loc="upper right")

In [ ]:
# Show the evolution of the alpha parameter during the learning process
plt.loglog(alpha_history_ex3)
plt.title("Alpha w.r.t iteration")
plt.ylabel("Alpha")
plt.xlabel("iteration")

### Evaluate Policy with Gymnasium

As a measure of performance, count the number of successful trials on 1000 episodes.

**Note**: Gymnasium considers the task is solved if you reach 76\% of success over the episodes.

In [ ]:
environment = gym.make("FrozenLake-v1", max_episode_steps=1000)

reward_list = []
NUM_EPISODES = 1000

for episode_index in range(NUM_EPISODES):
    state, info = environment.reset()
    episode_over = False

    while not episode_over:
        # action = epsilon_greedy_policy(state, q_array_ex3, epsilon)
        action = greedy_policy(state, q_array_ex3)
        state, reward, terminated, truncated, info = environment.step(action)
        episode_over = terminated or truncated

    reward_list.append(reward)

reward_df = pd.DataFrame(reward_list)

print(
    'Average reward (which is equivalent to a "success rate" in the FrozenLake environment as the total rewards in this environment are either 0 or 1):',
    np.average(reward_df),
)

environment.close()

### Test the optimized policy

In [ ]:
VIDEO_PREFIX_EX3 = "lab4_ex3_qlearning"

NUM_EPISODES = 3

file_path_list = [
    FIGS_DIR / f"{VIDEO_PREFIX_EX3}-episode-{episode_index}.mp4"
    for episode_index in range(NUM_EPISODES)
]

for file_path in file_path_list:
    file_path.unlink(missing_ok=True)

environment = gym.make("FrozenLake-v1", max_episode_steps=1000, render_mode="rgb_array")
environment = gym.wrappers.RecordVideo(
    environment,
    video_folder=str(FIGS_DIR),
    name_prefix=VIDEO_PREFIX_EX3,
    episode_trigger=lambda x: True
)
environment = gym.wrappers.RecordEpisodeStatistics(environment, buffer_length=NUM_EPISODES)

for episode_index in range(NUM_EPISODES):
    state, info = environment.reset()

    episode_over = False
    while not episode_over:
        # action = epsilon_greedy_policy(state, q_array_ex3, epsilon)
        action = greedy_policy(state, q_array_ex3)
        state, reward, terminated, truncated, info = environment.step(action)
        episode_over = terminated or truncated

    reward_list.append(reward)

print(f'Episode time taken: {environment.time_queue}')
print(f'Episode total rewards: {environment.return_queue}')
print(f'Episode lengths: {environment.length_queue}')

environment.close()

print("\nSelect the episode to play here 👇\n")

interact(video_selector, file_path=file_path_list);

### Question 3

Copy and paste the output of the following cell into the third question of the [Lab 4 - Evaluation](https://moodle.ip-paris.fr/mod/quiz/view.php?id=185078) in Moodle:  
*"What is the Q-value Q(s0,a0) of the initial state and action 0 ("Move Left") in the FrozenLake environment using the default parameters?"*

In [ ]:
# %%time

NUM_RUNS = 10

# Create the environment and set the maximum number of steps per episode
environment = gym.make("FrozenLake-v1", max_episode_steps=1000)

run_list = []
for _ in tqdm(range(1, NUM_RUNS)):
    q_array, _, _ = q_learning(environment, display=False)
    run_list.append(q_array[0][0].item())

environment.close()

print(f"\n\n⚠️ Copy and paste this value into Moodle 👉 {np.mean(run_list).item():.2f}\n(it has the required number of decimals)\n\n")

In [ ]:
# print(np.std(run_list))
# pd.DataFrame(run_list).plot.hist();